In [1]:
from google.generativeai import list_models
print(list_models())


<generator object list_models at 0x72544e312040>


/home/dulana-karunathilake/anaconda3/envs/agent2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import datetime
def get_system_time(format: str = "%Y-%m-%d %H:%M:%S" ):
    """ Returns the current date and time un the specified format"""
    current_time=datetime.datetime.now()
    formated_time = current_time.strftime(format)
    return formated_time

In [3]:
get_system_time()

'2025-12-06 14:16:01'

In [2]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()


generation_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a twitter techie influencer assistant tasked with writing excellent twitter posts."
            "Genarate the best twitter post possible for the user's request."
            "If the user provides critique, respond with a revised version of your previous attempts."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a viral twitter influencer grading a tweet. generate critique and recommendation for the user's tweet."
            "Always provide detailde recommendations, including requests for length, virality, style, etc"
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

generation_chain = generation_prompt | llm
reflection_chain = reflection_prompt | llm


ModuleNotFoundError: No module named 'langchain_core'

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_community.tools import TavilySearchResults
from langchain_core.prompts import PromptTemplate
import datetime

In [8]:

search_tool = TavilySearchResults(tavily_api_key="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")

print(search_tool)

api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********'))


In [8]:
from typing import List,Sequence
from dotenv import load_dotenv
from langchain_core.messages import BaseMessage,HumanMessage
from langgraph.graph import END,MessageGraph
from chains import generation_chain,reflection_chain

load_dotenv()
graph = MessageGraph()

REFLECT = "reflect"
GENERATE = "generate"

def generate_node(state):
    return generation_chain.invoke({
        "messages": state
    })

def reflection_node(messages):
    response = reflection_chain.invoke({
        "messages": messages
    })

    return [HumanMessage(content=response.content)]

graph.add_node(GENERATE,generate_node)
graph.add_node(REFLECT,reflection_node)


graph.set_entry_point(GENERATE)
 
def should_continue(state):
    if(len(state) > 2):
        return END
    return REFLECT

graph.add_conditional_edges(GENERATE,should_continue)
graph.add_edge(REFLECT,GENERATE)

app = graph.compile()
print(app.get_graph().draw_mermaid())
app.get_graph().print_ascii()

%%{init: {'flowchart': {'curve': 'linear'}}}%%
graph TD;
	__start__([<p>__start__</p>]):::first
	generate(generate)
	reflect(reflect)
	__end__([<p>__end__</p>]):::last
	__start__ --> generate;
	reflect --> generate;
	generate -.-> reflect;
	generate -.-> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

          +-----------+            
          | __start__ |            
          +-----------+            
                *                  
                *                  
                *                  
          +----------+             
          | generate |             
          +----------+             
          ...        ...           
         .              .          
       ..                ..        
+---------+           +---------+  
| reflect |           | __end__ |  
+---------+           +---------+  
